# Extract Datasets and Establish Benchmark

**Learning Objectives**
- Divide into Train, Evaluation and Test datasets
- Understand why we need each
- Pull data out of BigQuery and into CSV
- Establish Rules Based Benchmark

## Introduction 
In the previous notebook we demonstrated how to do ML in BigQuery. However BQML is limited to linear models.

For advanced ML we need to pull the data out of BigQuery and load it into a ML Framework, in our case TensorFlow.

While TensorFlow [can read from BigQuery directly](https://www.tensorflow.org/api_docs/python/tf/contrib/cloud/BigQueryReader), the performance is slow. The best practice is to first stage the BigQuery files as .csv files, and then read the .csv files into TensorFlow. 

The .csv files can reside on local disk if we're training locally, but if we're training in the cloud we'll need to move the .csv files to the cloud, in our case Google Cloud Storage.

### Set up environment variables and load necessary libraries

In [1]:
PROJECT = "vijays-sandbox"  # Replace with your PROJECT
REGION = "us-central1"            # Choose an available region for Cloud MLE

In [7]:
import os
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'service_account_key.json' # for local ONLY

## Review

In the [a_sample_explore_clean](a_sample_explore_clean.ipynb) notebook we came up with the following query to extract a repeatable and clean sample: 
<pre>
#standardSQL
SELECT
  (tolls_amount + fare_amount) AS fare_amount, -- label
  pickup_datetime,
  pickup_longitude, 
  pickup_latitude, 
  dropoff_longitude, 
  dropoff_latitude
FROM
  `nyc-tlc.yellow.trips`
WHERE
  -- Clean Data
  trip_distance > 0
  AND passenger_count > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  -- repeatable 1/5000th sample
  AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))),5000) = 1
  </pre>
  
We will use the same query **with one change**. Instead of using `pickup_datetime` as is, we will extract `dayofweek` and `hourofday` from it. This is to give us some categorical features in our dataset so we can illustrate how to deal with them when we get to feature engineering. The new query will be:

<pre>
SELECT
  (tolls_amount + fare_amount) AS fare_amount, -- label
  EXTRACT(DAYOFWEEK from pickup_datetime) AS dayofweek,
  EXTRACT(HOUR from pickup_datetime) AS hourofday,
  pickup_longitude, 
  pickup_latitude, 
  dropoff_longitude, 
  dropoff_latitude
-- rest same as before
</pre>

## Split into train, evaluation, and test sets

For ML modeling we need not just one, but three datasets.

**Train:** This is what our model learns on

**Evaluation (aka Validation):** We shouldn't evaluate our model on the same data we trained on because then we couldn't know whether it was memorizing the input data or whether it was generalizing. Therefore we evaluate on the evaluation dataset, aka validation dataset.

**Test:** We use our evaluation dataset to tune our hyperparameters (we'll cover hyperparameter tuning in a future lesson). We need to know that our chosen set of hyperparameters will work well for data we haven't seen before because in production, that will be the case. For this reason, we create a third dataset that we never use during the model development process. We only evaluate on this once our model development is finished. Data scientists don't always create a test dataset (aka holdout dataset), but to be thorough you should.

We can divide our existing 1/5000th sample three ways 70%/15%/15%  (or whatever split we like) with some modulo math demonstrated below.

Because we are using a hash function these results are deterministic, we'll get the same exact split every time the query is run (assuming the underlying data hasn't changed)

In [15]:
def create_query(phase, sample_size):
    basequery = """
    SELECT
        fare_amount,
        EXTRACT(DAYOFWEEK from pickup_datetime) AS dayofweek,
        EXTRACT(HOUR from pickup_datetime) AS hourofday,
        pickuplon,
        pickuplat,
        dropofflon,
        dropofflat,
        trips_last_5min
    FROM
        `vijays-sandbox.taxifare.traffic`
    WHERE
        trip_distance > 0
        AND fare_amount >= 2.5
        AND pickuplon > -78
        AND pickuplon < -70
        AND dropofflon > -78
        AND dropofflon < -70
        AND pickuplat > 37
        AND pickuplat < 45
        AND dropofflat > 37
        AND dropofflat < 45
        AND passenger_count > 0
        AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), EVERY_N) = 1
    """

    if phase == "TRAIN":
        subsample = """
        AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), EVERY_N * 100) >= (EVERY_N * 0)
        AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), EVERY_N * 100) <  (EVERY_N * 70)
        """
    elif phase == "VALID":
        subsample = """
        AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), EVERY_N * 100) >= (EVERY_N * 70)
        AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), EVERY_N * 100) <  (EVERY_N * 85)
        """
    elif phase == "TEST":
        subsample = """
        AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), EVERY_N * 100) >= (EVERY_N * 85)
        AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), EVERY_N * 100) <  (EVERY_N * 100)
        """

    query = basequery + subsample
    return query.replace("EVERY_N", sample_size)

## Write to CSV
Now let's execute a query for train/valid/test and write the results to disk in csv format. We use Pandas's `.to_csv()` method to do so.

In [16]:
from google.cloud import bigquery
bq = bigquery.Client(project=PROJECT)

for phase in ["TRAIN", "VALID", "TEST"]:
    # 1. Create query string
    query_string = create_query(phase, "5000")
    # 2. Load results into DataFrame
    df = bq.query(query_string).to_dataframe()

    # 3. Write DataFrame to CSV
    df.to_csv("taxi-{}.csv".format(phase.lower()), index_label = False, index = False)
    print("Wrote {} lines to {}".format(len(df), "taxi-{}.csv".format(phase.lower())))

Wrote 151615 lines to taxi-train.csv
Wrote 31820 lines to taxi-valid.csv
Wrote 34080 lines to taxi-test.csv


Note that even with a 1/5000th sample we have a good amount of data for ML. 150K training examples and 30K validation.

### Verify that datasets exist 

In [17]:
!ls -l *.csv

-rw-r--r--  1 reddyv  google  1967394 Jul 29 13:51 taxi-test.csv
-rw-r--r--  1 reddyv  google  8734343 Jul 29 13:49 taxi-train.csv
-rw-r--r--  1 reddyv  google  1830429 Jul 29 13:50 taxi-valid.csv


### Preview one of the files

In [18]:
!head taxi-train.csv

fare_amount,dayofweek,hourofday,pickuplon,pickuplat,dropofflon,dropofflat,trips_last_5min
6.1,2,0,-73.986895,40.729723,-74.00631,40.739407,1129
9.7,7,0,-73.945783,40.777807,-73.97539,40.757712,2876
5.3,6,0,-74.00644,40.739349,-73.999379,40.731804,3950
7.3,5,0,-73.966118,40.753983,-73.945605,40.782802,1334
6.5,7,0,-73.974153,40.762767,-73.989152,40.742727,2623
22.9,1,0,-73.977188,40.774063,-73.962647,40.654768,2833
22.9,2,0,-74.00188,40.745947,-73.968497,40.639375,2002
6.1,3,0,-73.994051,40.751077,-73.977333,40.778875,661
5.3,5,0,-73.980898,40.744515,-73.973383,40.753497,1938


Looks good! We now have our ML datasets and are ready to train ML models, validate them and test them.